### Hello World with KNative

In [1]:
%cd ./helloworld-python


/Users/i033085/gitrepos/Knative/helloworld-python


In [4]:
! kubectl apply -f service.yaml

Error from server (InternalError): error when creating "service.yaml": Internal error occurred: admission webhook "webhook.serving.knative.dev" denied the request: mutation failed: expected exactly one, got neither: spec.runLatest.configuration.revisionTemplate.spec.volumes[0].configMap, spec.runLatest.configuration.revisionTemplate.spec.volumes[0].secret


In [3]:
! kubectl get svc istio-ingressgateway --namespace istio-system

NAME                   TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)                                                                                                                   AGE
istio-ingressgateway   NodePort   10.108.14.171   <none>        80:31380/TCP,443:31390/TCP,31400:31400/TCP,15011:30638/TCP,8060:31749/TCP,853:31693/TCP,15030:31520/TCP,15031:31924/TCP   34d


In [4]:
! kubectl get ksvc helloworld-python  --output=custom-columns=NAME:.metadata.name,DOMAIN:.status.domain

NAME                DOMAIN
helloworld-python   helloworld-python.default.example.com


In [6]:
! curl -H "Host: helloworld-python.default.example.com" http://localhost:31380

Hello Python Sample v2!


In [6]:
! kubectl -n knative-serving get cm config-autoscaler --output json

{
    "apiVersion": "v1",
    "data": {
        "_example": "################################\n#                              #\n#    EXAMPLE CONFIGURATION     #\n#                              #\n################################\n\n# This block is not actually functional configuration,\n# but serves to illustrate the available configuration\n# options and document them in a way that is accessible\n# to users that `kubectl edit` this config map.\n#\n# These sample configuration options may be copied out of\n# this block and unindented to actually change the configuration.\n\n# The Revision ContainerConcurrency field specifies the maximum number\n# of requests the Container can handle at once. Container concurrency\n# target percentage is how much of that maximum to use in a stable\n# state. E.g. if a Revision specifies ContainerConcurrency of 10, then\n# the Autoscaler will try to maintain 7 concurrent connections per pod\n# on average. A value of 0.7 is chosen because the Autoscaler p

### Role out a new Revision

In [7]:
! kubectl apply -f service-rev1.yaml

service.serving.knative.dev "helloworld-python" configured


In [9]:
! curl -H "Host: helloworld-python.default.example.com" http://localhost:31380

Hello Python Sample v3!


In [10]:
! kubectl get revisions

NAME                      AGE
helloworld-python-btj9b   28s
helloworld-python-kqd68   7m


In [11]:
! kubectl get route --output yaml

apiVersion: v1
items:
- apiVersion: serving.knative.dev/v1alpha1
  kind: Route
  metadata:
    creationTimestamp: 2019-04-02T05:43:58Z
    finalizers:
    - routes.serving.knative.dev
    generation: 1
    labels:
      serving.knative.dev/service: helloworld-python
    name: helloworld-python
    namespace: default
    ownerReferences:
    - apiVersion: serving.knative.dev/v1alpha1
      blockOwnerDeletion: true
      controller: true
      kind: Service
      name: helloworld-python
      uid: 4fb3239a-550a-11e9-a3c9-025000000001
    resourceVersion: "1115381"
    selfLink: /apis/serving.knative.dev/v1alpha1/namespaces/default/routes/helloworld-python
    uid: 4fbe2d46-550a-11e9-a3c9-025000000001
  spec:
    traffic:
    - configurationName: helloworld-python
      percent: 100
  status:
    address:
      hostname: helloworld-python.default.svc.cluster.local
    conditions:
    - lastTransitionTime: 2019-04-02T05:44:08Z
      status: "True"
      type: AllTrafficAssigned
    - lastT

In [12]:
! kubectl apply -f service-rev2.yaml

service.serving.knative.dev "helloworld-python" configured


In [13]:
! kubectl get route --output yaml

apiVersion: v1
items:
- apiVersion: serving.knative.dev/v1alpha1
  kind: Route
  metadata:
    creationTimestamp: 2019-04-02T05:43:58Z
    finalizers:
    - routes.serving.knative.dev
    generation: 2
    labels:
      serving.knative.dev/service: helloworld-python
    name: helloworld-python
    namespace: default
    ownerReferences:
    - apiVersion: serving.knative.dev/v1alpha1
      blockOwnerDeletion: true
      controller: true
      kind: Service
      name: helloworld-python
      uid: 4fb3239a-550a-11e9-a3c9-025000000001
    resourceVersion: "1115682"
    selfLink: /apis/serving.knative.dev/v1alpha1/namespaces/default/routes/helloworld-python
    uid: 4fbe2d46-550a-11e9-a3c9-025000000001
  spec:
    traffic:
    - name: current
      percent: 50
      revisionName: helloworld-python-kqd68
    - name: candidate
      percent: 50
      revisionName: helloworld-python-btj9b
    - configurationName: helloworld-python
      name: latest
      percent: 0
  status:
    address:
   

In [15]:
! curl -H "Host: helloworld-python.default.example.com" http://localhost:31380

Hello Python Sample v2!


In [40]:
! hey -z 30s -c 50 \
  -host "helloworld-python.default.example.com" \
  "http://localhost:31380?sleep=100" \

panic: runtime error: invalid memory address or nil pointer dereference
[signal SIGSEGV: segmentation violation code=0x1 addr=0x0 pc=0x1556e89]

goroutine 1 [running]:
main.main()
	/Users/i033085/golang/src/github.com/sbcd90/hey/hey.go:244 +0x4e9


In [24]:
! kubectl delete -f service.yaml

service.serving.knative.dev "helloworld-python" deleted


### Auto Scaling

In [ ]:
%cd /Users/i033085/externalRepos/docs/serving/samples/autoscale-go

In [ ]:
! kubectl apply -f service.yaml

In [ ]:
! curl --header "Host: autoscale-go.default.example.com" "http://localhost:31380?prime=10000&bloat=5"

In [ ]:
! hey -z 30s -c 50 \
  -host "autoscale-go.default.example.com" \
  "http://localhost:31380?prime=10000&bloat=5" \
  && kubectl get pods

In [ ]:
! kubectl delete -f service.yaml

In [ ]:
! kubectl get pods

### Deploy gRPC model Server on KNative

In [16]:
%cd /Users/i033085/gitrepos/mlf-python-grpc-model-server/knative

/Users/i033085/gitrepos/mlf-python-grpc-model-server/knative


In [17]:
!kubectl apply -f sample.yaml

service.serving.knative.dev "mlf-python-grpc-model-server" created


In [18]:
%cd /Users/i033085/gitrepos/mlf-python-grpc-model-server/test

/Users/i033085/gitrepos/mlf-python-grpc-model-server/test


In [19]:
!python python_grpc_client.py 

Traceback (most recent call last):
  File "python_grpc_client.py", line 2, in <module>
    from tensorflow_serving.apis import predict_pb2 as tensorflow__serving_dot_apis_dot_predict__pb2
ModuleNotFoundError: No module named 'tensorflow_serving'


In [20]:
!hey -n 1000 -c 100 -M tensorflow-feature-sum-model -v 1 -k X -d 1,2,3,4 -server-host-override mlf-python-grpc-model-server.default.example.com -insecure -g localhost:31380


Summary:
  Total:	45.8586 secs
  Slowest:	19.0728 secs
  Fastest:	0.0314 secs
  Average:	4.3434 secs
  Requests/sec:	21.8062
  
  Total data:	21000 bytes
  Size/request:	21 bytes

Response time histogram:
  0.031 [1]	|
  1.936 [550]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
  3.840 [76]	|■■■■■■
  5.744 [62]	|■■■■■
  7.648 [100]	|■■■■■■■
  9.552 [19]	|■
  11.456 [24]	|■■
  13.360 [20]	|■
  15.265 [66]	|■■■■■
  17.169 [62]	|■■■■■
  19.073 [20]	|■


Latency distribution:
  10% in 0.0997 secs
  25% in 0.2301 secs
  50% in 1.0633 secs
  75% in 6.3283 secs
  90% in 14.7917 secs
  95% in 16.1972 secs
  99% in 18.3233 secs

Details (average, fastest, slowest):
  DNS+dialup:	0.0562 secs, 0.0314 secs, 19.0728 secs
  DNS-lookup:	0.0000 secs, 0.0000 secs, 0.0000 secs
  req write:	0.0000 secs, 0.0000 secs, 0.0000 secs
  resp wait:	0.0000 secs, 0.0000 secs, 0.0000 secs
  resp read:	0.0000 secs, 0.0000 secs, 0.0000 secs

Status code distribution:
  [200]	1000 responses





In [21]:
%cd /Users/i033085/gitrepos/mlf-python-grpc-model-server/knative

/Users/i033085/gitrepos/mlf-python-grpc-model-server/knative


In [22]:
! kubectl delete -f sample.yaml

service.serving.knative.dev "mlf-python-grpc-model-server" deleted
